# Collective progress

**Example notebook for creating anonymised, collective information on progress**

* Before running this notebook, you need to prepare the data you want to assess. To do so, please use the notebooks  "prepare-PRIMAP-hist-data-for-collective-progress-plots.ipynb" or "prepare-PRIMAP-data.ipynb" in case the data you need is not yet available.
* For testing, some example data is available in the folder "proc_data"
* Enter the name of the file that you wish to use in the second cell and some of the plotting parameters. After that you can run the full notebook with minimal changes. 

In [1]:
# import modules

# system 
import re
import os

# calculation
import pandas as pd
import numpy as np

# plotting
%matplotlib inline
import seaborn
import matplotlib

# global stocktake tools
from gst_tools.make_plots import *
import gst_tools.gst_utils as utils

In [2]:
from parameters import *

In [3]:
# USER INPUT

# First, choose which file you want to plot the data for
data_file_name = proc_data_file#'PRIMAP-hist_v2.3.1_CO2-total-excl-LU.csv'


# other options include...
# 'PRIMAP-hist_v2.0_KyotoGHG-AR4-total-excl-LU.csv'
# 'UN-population-data-2017.csv'
# 'PRIMAP-hist_v2.0_Energy-CO2.csv'
# 'PRIMAP-hist_UN-2017_calc__CO2-per-population.csv'
# 'PRIMAP-hist_UN-2017_calc_CO2-total-excl-LU-per-population.csv'
# 'WDI2017_GDP-PPP.csv'

# Second, choose which years you are interested in analysing
#years_of_interest = ['1990', '2005', '2016']

# Third, update data description display names!
# TODO - default to automatic if not specified.
#variable_name_to_display = 'Energy CO2 emissions'#'Total CO2 emissions excl. LULUCF'
#data_source_to_display = #'PRIMAP-hist'

# Save plots?
# Set the following to True if plots should be saved. 
# If False, plots will be shown on screen but not saved to a file.
#save_opt = True

In [4]:
# DATA READING AND PREP

# read the data from file 
fname_in = os.path.join('proc-data', data_file_name)
data = pd.read_csv(fname_in)

# Check the data format
if not utils.verify_data_format(data):          # WARNING: Script from Utils used here.
    print('WARNING: The data is not correctly formatted! Please check before continuing!')

# extract the key information
variable = data['variable'].unique()[0]
unit = data['unit'].unique()[0]

# tidy up for next steps
data_years = utils.set_countries_as_index(data)
data_years = data_years.dropna(axis=1, how='all')
data_years = data_years.dropna(axis=0, how='any')

# remove comment below to display the data
data_years

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
country,,,,,,,,,,,,,,,,,,,,,
AFG,0.00369,0.00392,0.00421,0.00455,0.00487,0.00515,0.00537,0.00554,0.00568,0.00585,...,0.00868,0.00895,0.00925,0.00956,0.00987,0.01020,0.01040,0.01070,0.01100,0.01120
AGO,0.24500,0.25300,0.26100,0.27000,0.27900,0.28800,0.29600,0.30400,0.31200,0.32100,...,0.47100,0.48800,0.50600,0.52400,0.54300,0.56200,0.58100,0.60000,0.62000,0.63100
ALB,0.03610,0.03610,0.03570,0.03510,0.03460,0.03420,0.03400,0.03410,0.03420,0.03430,...,0.03240,0.03220,0.03210,0.03210,0.03220,0.03220,0.03220,0.03230,0.03230,0.03160
AND,0.06630,0.06700,0.07080,0.07500,0.07800,0.08210,0.08620,0.08400,0.03710,0.02420,...,0.01180,0.01060,0.01050,0.00974,0.00909,0.00920,0.00904,0.00892,0.00895,0.00621
ARE,0.28000,0.29600,0.31400,0.33200,0.35000,0.36800,0.38600,0.40600,0.42700,0.44900,...,1.24000,1.30000,1.34000,1.35000,1.36000,1.38000,1.39000,1.41000,1.43000,1.63000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSM,0.00332,0.00334,0.00337,0.00340,0.00344,0.00346,0.00349,0.00351,0.00352,0.00354,...,0.00379,0.00382,0.00385,0.00388,0.00391,0.00394,0.00397,0.00400,0.00402,0.00405
YEM,1.81000,1.90000,2.00000,2.11000,2.21000,2.30000,2.39000,2.47000,2.54000,2.61000,...,3.55000,3.64000,3.74000,3.84000,3.94000,4.05000,4.15000,4.25000,4.35000,4.43000
ZAF,4.18000,4.41000,12.20000,10.20000,10.50000,11.20000,15.40000,15.30000,13.70000,11.60000,...,2.77000,2.15000,3.24000,3.60000,2.91000,2.22000,2.24000,2.25000,2.27000,1.17000


In [5]:
variable_name_to_display

'N2O emissions from IPPU'

In [6]:
# Plot 1 - make a histogram of absolute data

# how should the xaxis be labelled?
xaxlabel = variable_name_to_display

# make a plot for each year
for selected_year in years_of_interest:
    
    title = 'Distribution of ' + variable_name_to_display + ' in ' + str(selected_year)
    
    make_histogram(data_years[selected_year], unit,             # WARNING: This is a script from the make_plots.py tool.
                   xlabel=xaxlabel, title=title, 
                   sourcename=data_source_to_display,
                   remove_outliers=True, ktuk=3,
                   save_plot=save_opt, plot_name=(variable + '-' + 'absolute' + '-' + str(selected_year)))


---------
Making  N2O_emissions_from_IPPU-absolute-1990 plot.
---------
-----------
Identifying and removing outliers
lower outliers are:
Series([], Name: 1990, dtype: float64)
upper outliers are: 
country
BEL     13.50
BRA     12.30
CAN     38.50
CHN     15.40
DEU     78.50
ESP     10.90
EUU    314.00
FRA     80.00
GBR     81.90
HUN     10.80
IRN      9.54
ITA     24.20
JPN     33.30
NLD     24.50
POL     12.30
ROU     13.90
RUS     15.90
TUR     11.70
UKR     19.00
USA    112.00
Name: 1990, dtype: float64
---
bins set to range(0, 9)


c:\Users\EduardoPosada\AppData\Local\Programs\Python\Python310\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


---------
Making  N2O_emissions_from_IPPU-absolute-2005 plot.
---------
-----------
Identifying and removing outliers
lower outliers are:
Series([], Name: 2005, dtype: float64)
upper outliers are: 
country
AUS      9.41
BEL     11.40
BRA     24.80
CAN     13.90
CHN    143.00
DEU     28.90
EGY     10.60
ESP      8.92
EUU    191.00
FRA     22.70
GBR     12.30
IND     22.30
IRN     38.60
ITA     27.70
JPN      9.82
KOR     35.80
LTU      8.02
NLD     22.80
POL     15.30
ROU     10.00
RUS     16.20
TUR     12.60
UKR     12.60
USA     83.70
Name: 2005, dtype: float64
---
bins set to range(0, 9)
---------
Making  N2O_emissions_from_IPPU-absolute-2016 plot.
---------
-----------
Identifying and removing outliers
lower outliers are:
Series([], Name: 2016, dtype: float64)
upper outliers are: 
country
AUS      5.29
BEL      3.65
CHN    312.00
DEU      3.69
DZA      4.09
EGY     18.60
EUU     33.50
FRA      3.45
IDN      4.15
IND     11.40
IRN     14.90
IRQ      5.59
ISR      3.79
JPN      3.71
N

In [7]:
data_years

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
country,,,,,,,,,,,,,,,,,,,,,
AFG,0.00369,0.00392,0.00421,0.00455,0.00487,0.00515,0.00537,0.00554,0.00568,0.00585,...,0.00868,0.00895,0.00925,0.00956,0.00987,0.01020,0.01040,0.01070,0.01100,0.01120
AGO,0.24500,0.25300,0.26100,0.27000,0.27900,0.28800,0.29600,0.30400,0.31200,0.32100,...,0.47100,0.48800,0.50600,0.52400,0.54300,0.56200,0.58100,0.60000,0.62000,0.63100
ALB,0.03610,0.03610,0.03570,0.03510,0.03460,0.03420,0.03400,0.03410,0.03420,0.03430,...,0.03240,0.03220,0.03210,0.03210,0.03220,0.03220,0.03220,0.03230,0.03230,0.03160
AND,0.06630,0.06700,0.07080,0.07500,0.07800,0.08210,0.08620,0.08400,0.03710,0.02420,...,0.01180,0.01060,0.01050,0.00974,0.00909,0.00920,0.00904,0.00892,0.00895,0.00621
ARE,0.28000,0.29600,0.31400,0.33200,0.35000,0.36800,0.38600,0.40600,0.42700,0.44900,...,1.24000,1.30000,1.34000,1.35000,1.36000,1.38000,1.39000,1.41000,1.43000,1.63000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSM,0.00332,0.00334,0.00337,0.00340,0.00344,0.00346,0.00349,0.00351,0.00352,0.00354,...,0.00379,0.00382,0.00385,0.00388,0.00391,0.00394,0.00397,0.00400,0.00402,0.00405
YEM,1.81000,1.90000,2.00000,2.11000,2.21000,2.30000,2.39000,2.47000,2.54000,2.61000,...,3.55000,3.64000,3.74000,3.84000,3.94000,4.05000,4.15000,4.25000,4.35000,4.43000
ZAF,4.18000,4.41000,12.20000,10.20000,10.50000,11.20000,15.40000,15.30000,13.70000,11.60000,...,2.77000,2.15000,3.24000,3.60000,2.91000,2.22000,2.24000,2.25000,2.27000,1.17000


In [8]:
# Plot 2 - trends

# Calculate trends from the absolute data
# trends - % change in any given year
# rolling_trends - % annual change averaged over the specified number of years (here 5 is recommended)
# trends_unit - unit of the trend, here % change
trends, rolling_trends, trends_unit = utils.calculate_trends(data_years, num_years_trend=5)     #WARNING: Utils here.

# define some labels for the plots
trends_variable = 'average annual change'
thistitle = "5-year rolling average trend in \n" + variable_name_to_display + "\nin " + str(data_years.columns[-1])

# make a plot showing the trend in the final year of available data
make_histogram(rolling_trends.iloc[:,-1], trends_unit, 
               xlabel=trends_variable,
               title=thistitle,
               remove_outliers=True, ktuk=2,
               sourcename=data_source_to_display, 
               save_plot=save_opt, plot_name=(variable + '-' + 'rolling-average'))


Averaging trend over 5 years.
---------
Making  N2O_emissions_from_IPPU-rolling-average plot.
---------
-----------
Identifying and removing outliers
lower outliers are:
country
AND    -6.426929
ARG    -4.931499
BEL    -7.958416
BRA    -7.206811
CHL   -36.967076
CZE    -6.453793
IND    -4.858020
ISR    -6.601369
JPN    -9.907701
KOR   -16.963587
LTU   -10.479306
MCO   -13.415309
NOR    -6.533765
PHL   -18.470159
ROU   -19.542926
SMR    -9.845380
SRB    -8.434673
SVK    -6.888433
ZAF   -13.986654
Name: 2019, dtype: float64
upper outliers are: 
country
AUS      9.491836
BGR    103.275086
CAN      7.835506
CHE      8.026181
IDN     41.870159
MLT     37.016279
NZL      8.255660
SVN     59.626123
Name: 2019, dtype: float64
---
bins set to range(-9, 9)


c:\Users\EduardoPosada\AppData\Local\Programs\Python\Python310\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [9]:
# Plot 3 - change since year X

# run calculations - dataframe of differences in all years relative to the specified year
# the function returns both absolute and relative (%) values
df_abs_diff_1990, df_perc_diff_1990 = utils.calculate_diff_since_yearX(data_years, '1990')
df_abs_diff_2005, df_perc_diff_2005 = utils.calculate_diff_since_yearX(data_years, '2005')

# make plots

# a few selected years; difference from 1990
for selected_year in years_of_interest:
    make_histogram(df_perc_diff_1990[selected_year], "%", 
                   xlabel='change since 1990', 
                   title=('change in ' + variable_name_to_display + '\n from 1990 to ' + str(selected_year)), 
                   sourcename=data_source_to_display,
                   remove_outliers=True, ktuk=3, 
                   save_plot=save_opt, plot_name=(variable + '-' + 'change-since-1990' + '-in-' + str(selected_year)))

    
# decrease in the last year as compared to 2005 
make_histogram(df_perc_diff_2005.iloc[:,-1], '%', 
               xlabel='change since 2005', 
               title=('change in ' + variable_name_to_display + '\n from 2005 to ' + str(df_perc_diff_2005.columns[-1])), 
               sourcename=data_source_to_display,
               remove_outliers=False, 
               save_plot=save_opt, plot_name=(variable + '-' + 'change-since-2005'))

# same plot, but removing outliers to show difference...
make_histogram(df_perc_diff_2005.iloc[:,-1], '%', 
               xlabel='change since 2005', 
               title=('change in ' + variable_name_to_display + '\n from 2005 to ' + str(df_perc_diff_2005.columns[-1])), 
               sourcename=data_source_to_display,
               remove_outliers=True, 
               save_plot=save_opt, plot_name=(variable + '-' + 'change-since-2005-excl-outliers'))


Calculating difference compared to 1990
Calculating difference compared to 2005
---------
Making  N2O_emissions_from_IPPU-change-since-1990-in-1990 plot.
---------
---------
All values in the series are the same! Exiting plotting routine for N2O_emissions_from_IPPU-change-since-1990-in-1990
---------
---------
Making  N2O_emissions_from_IPPU-change-since-1990-in-2005 plot.
---------
-----------
Identifying and removing outliers
lower outliers are:
Series([], Name: 2005, dtype: float64)
upper outliers are: 
country
CHN     828.571429
IND     264.975450
IRN     304.612159
KOR    4223.671498
THA     707.106599
TKM     273.770492
Name: 2005, dtype: float64
---
bins set to range(-191, 191)
---------
Making  N2O_emissions_from_IPPU-change-since-1990-in-2016 plot.
---------
-----------
Identifying and removing outliers
lower outliers are:
Series([], Name: 2016, dtype: float64)
upper outliers are: 
country
CHN    1925.974026
IDN     986.387435
THA     920.304569
Name: 2016, dtype: float64
---


## Below here is space for code for testing and debugging!

In [ ]:
# show example data
#data.columns


Index(['category', 'country', 'scenario', 'source', 'unit', 'variable', '1990',
       '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999',
       '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019'],
      dtype='object')

In [ ]:
#data_1 = data
#data_1 = data_1.dropna(axis=1, how='all')
#data_1 = data_1.dropna(axis=0, how='any')
#data_1

,category,country,scenario,source,unit,variable,1990,1991,1992,1993,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,M.0.EL,AFG,HISTCR,PRIMAP-hist_v2.3.1,Gg CO2 / yr,CO2-total-excl-LU,2810.0,2640.0,1590.0,1520.0,...,8650.0,12400.0,10500.0,8830.0,8140.0,8230.0,7100.0,6860.0,7060.0,9780.0
1,M.0.EL,AGO,HISTCR,PRIMAP-hist_v2.3.1,Gg CO2 / yr,CO2-total-excl-LU,10800.0,11200.0,11600.0,12000.0,...,23500.0,24700.0,26500.0,26300.0,38000.0,29100.0,30300.0,29600.0,27600.0,27000.0
2,M.0.EL,ALB,HISTCR,PRIMAP-hist_v2.3.1,Gg CO2 / yr,CO2-total-excl-LU,4270.0,6050.0,4030.0,3920.0,...,5900.0,6660.0,6390.0,6590.0,7300.0,6220.0,6210.0,7160.0,6670.0,7090.0
3,M.0.EL,AND,HISTCR,PRIMAP-hist_v2.3.1,Gg CO2 / yr,CO2-total-excl-LU,409.0,417.0,426.0,436.0,...,556.0,525.0,523.0,512.0,497.0,497.0,504.0,506.0,494.0,470.0
4,M.0.EL,ARE,HISTCR,PRIMAP-hist_v2.3.1,Gg CO2 / yr,CO2-total-excl-LU,47000.0,52700.0,53300.0,61000.0,...,154000.0,162000.0,199000.0,197000.0,191000.0,211000.0,203000.0,171000.0,170000.0,170000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,M.0.EL,WSM,HISTCR,PRIMAP-hist_v2.3.1,Gg CO2 / yr,CO2-total-excl-LU,104.0,108.0,111.0,111.0,...,188.0,203.0,199.0,199.0,210.0,236.0,247.0,245.0,252.0,353.0
192,M.0.EL,YEM,HISTCR,PRIMAP-hist_v2.3.1,Gg CO2 / yr,CO2-total-excl-LU,9710.0,9320.0,10100.0,8810.0,...,22400.0,22800.0,23400.0,31700.0,30700.0,16400.0,13200.0,12700.0,20300.0,20600.0
193,M.0.EL,ZAF,HISTCR,PRIMAP-hist_v2.3.1,Gg CO2 / yr,CO2-total-excl-LU,274000.0,284000.0,262000.0,279000.0,...,447000.0,461000.0,443000.0,455000.0,485000.0,460000.0,464000.0,472000.0,456000.0,465000.0
194,M.0.EL,ZMB,HISTCR,PRIMAP-hist_v2.3.1,Gg CO2 / yr,CO2-total-excl-LU,2490.0,2450.0,2480.0,2530.0,...,2650.0,2880.0,3600.0,3880.0,4270.0,4360.0,4730.0,6500.0,7030.0,7090.0


In [ ]:
#data.loc[data['country'] == 'AFG']


,category,country,scenario,source,unit,variable,1990,1991,1992,1993,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,M.0.EL,AFG,HISTCR,PRIMAP-hist_v2.3.1,Gg CO2 / yr,CO2-total-excl-LU,2810.0,2640.0,1590.0,1520.0,...,8650.0,12400.0,10500.0,8830.0,8140.0,8230.0,7100.0,6860.0,7060.0,9780.0
